In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

y = data['Price']

melb_predictions = data.drop(['Price'], axis=1)
X = melb_predictions.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(preds, y_valid)

#### Score from Approach 1 (Drop Columns with Missing Values)

In [4]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print(f'MAE from Approach 1 (Drop columns with missing values):')
print(f'{score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid):.0f}')

MAE from Approach 1 (Drop columns with missing values):
183550


#### Score from Approach 2 (Imputation)

In [14]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

imputed_X_train.index = X_train.index
imputed_X_valid.index = X_valid.index

print('MAE from Approach 2 (Imputation)')
print(f'{score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid):.0f}')

MAE from Approach 2 (Imputation)
178166


In [15]:
imputed_X_valid.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
8505,4.0,8.0,3016.0,4.0,2.0,2.0,450.0,190.000000,1910.000000,-37.86100,144.89850,6380.0
5523,2.0,6.6,3011.0,2.0,1.0,0.0,172.0,81.000000,1900.000000,-37.81000,144.88960,2417.0
12852,3.0,10.5,3020.0,3.0,1.0,1.0,581.0,153.764119,1964.839866,-37.76740,144.82421,4217.0
4818,3.0,4.5,3181.0,2.0,2.0,1.0,128.0,134.000000,2000.000000,-37.85260,145.00710,7717.0
12812,3.0,8.5,3044.0,3.0,2.0,2.0,480.0,153.764119,1964.839866,-37.72523,144.94567,7485.0


In [16]:
X_valid.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
8505,4,8.0,3016.0,4.0,2.0,2.0,450.0,190.0,1910.0,-37.86100,144.89850,6380.0
5523,2,6.6,3011.0,2.0,1.0,0.0,172.0,81.0,1900.0,-37.81000,144.88960,2417.0
12852,3,10.5,3020.0,3.0,1.0,1.0,581.0,NaN,NaN,-37.76740,144.82421,4217.0
4818,3,4.5,3181.0,2.0,2.0,1.0,128.0,134.0,2000.0,-37.85260,145.00710,7717.0
12812,3,8.5,3044.0,3.0,2.0,2.0,480.0,NaN,NaN,-37.72523,144.94567,7485.0


#### Score from Approach 3 (An Extension to imputation)

In [7]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed columns names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print('MAE from Approach 3 (An extension to imputation)')
print(f'{score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid):.0f}')

MAE from Approach 3 (An extension to imputation)
178928


In [8]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
